<a href="https://colab.research.google.com/github/currencyfxjle/An-lisis-Causal-y-Predictivo-Utilizando-Regresi-n/blob/main/Kanpsack__GDF_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 3.9 MB/s eta 0:00:00


# **Full Project Allocation**

In [3]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# ------------------------------------------------
# 1. Fetch and Clean Supply Data
# ------------------------------------------------
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y'].copy()
    supply_df['Est Cash Amount'] = pd.to_numeric(
        supply_df['Est Cash Amount'].replace({',': ''}, regex=True),
        errors='coerce'
    )
    supply_df['Project_date'] = pd.to_datetime(
        supply_df['Project_date'], errors='coerce'
    )

    # Convert GDF Profit to numeric (e.g., '25%' to 0.25)
    supply_df['GDF Profit'] = (
        supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    )
    supply_df = supply_df.dropna()

    # ------------------------------------------------
    # **NEW:** Ensure Priority is numeric
    # ------------------------------------------------
    supply_df['Priority'] = pd.to_numeric(supply_df['Priority'], errors='coerce')
    supply_df = supply_df.dropna(subset=['Priority'])  # Remove rows with invalid Priority

    # -------------------------------
    # **UPDATED SORTING:** Sort by Priority ASC, GDF Profit DESC,
    # then Est Cash Amount DESC, then Project_date ASC
    # -------------------------------
    supply_df = supply_df.sort_values(
        by=['Priority', 'GDF Profit', 'Est Cash Amount', 'Project_date'],
        ascending=[True, False, False, True]
    )

    # Add additional tracking columns
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Add category based on Est Cash Amount
    supply_df['Project Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Include project state for info
    supply_df['Project State'] = supply_df['State']

    return supply_df

# ------------------------------------------------
# 2. Fetch and Clean Demand Data
# ------------------------------------------------
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Rename 'purchaser_name' to 'Purchaser Name' for consistency
    demand_df = demand_df.rename(columns={'purchaser_name': 'Purchaser Name'})

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(
        demand_df['approximated_appetite'].replace({',': ''}, regex=True),
        errors='coerce'
    )
    demand_df['pending_amount'] = pd.to_numeric(
        demand_df['pending_amount'].replace({',': ''}, regex=True),
        errors='coerce'
    )
    demand_df['Purchaser_Date'] = pd.to_datetime(
        demand_df['Purchaser_Date'], errors='coerce'
    )
    demand_df = demand_df.dropna()

    # Track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite'].astype(float)

    # Ensure 'pending_amount' is float
    demand_df['pending_amount'] = demand_df['pending_amount'].astype(float)

    # Add category based on approximated appetite
    demand_df['Purchaser Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Add priority for state (CA=1, others=2)
    demand_df['State Priority'] = demand_df['State'].apply(
        lambda x: 1 if x == 'CA' else 2
    )

    # (Optional) Sort demand initially by appetite desc, then state priority
    demand_df = demand_df.sort_values(
        by=['approximated_appetite', 'State Priority'],
        ascending=[False, True]
    )

    return demand_df

# ------------------------------------------------
# 3. Match Supply to Demand without Financing
# ------------------------------------------------
def match_supply_to_demand(supply_df, demand_df):
    """
    Allocates chunks from supply to purchasers in demand.
    Prioritizes Priority (ascending),
    then GDF Profit (descending),
    then Est Cash Amount (descending) on supply side,
    and CA-first + large appetites on the demand side.

    Only allocates if purchaser's remaining appetite >= chunk cost.
    """
    allocations = []
    total_allocated = 0.0

    # Group supply by Project Name in the sorted order
    grouped_supply = supply_df.groupby('Project Name', sort=False)

    for project_name, group in grouped_supply:
        # If the entire project is already allocated, skip
        if group['Fully Allocated'].all():
            continue

        project_total_cost = group['Est Cash Amount'].sum()
        project_chunks = group.to_dict('records')  # Convert group to list of chunk-rows

        can_allocate_project = True
        temp_allocations = []
        temp_demand = demand_df.copy()  # Work on a copy, finalize only if successful

        # -------------------------------------
        # For each chunk in the current project
        # -------------------------------------
        for chunk in project_chunks:
            if chunk['Fully Allocated']:
                continue  # Already allocated

            chunk_cost = chunk['Est Cash Amount']
            allocated = False

            # Sort potential purchasers by: CA first, then large appetite
            # (State Priority asc => CA=1 first, then appetite desc)
            purchasers = temp_demand[temp_demand['remaining_appetite'] >= chunk_cost].sort_values(
                by=['State Priority', 'remaining_appetite'],
                ascending=[True, False]
            ).to_dict('records')

            # -------------------------------------
            # Try each purchaser in sorted order
            # -------------------------------------
            for purchaser in purchasers:
                # Must meet date requirement
                if purchaser['Purchaser_Date'] > chunk['Project_date']:
                    # This purchaser isn't eligible yet
                    continue

                # Current stats
                p_appetite = purchaser['remaining_appetite']
                p_pending = purchaser['pending_amount']

                # Allocate the full chunk if possible
                new_appetite = p_appetite - chunk_cost
                new_pending = p_pending - chunk_cost

                # Clamp so they never go below zero
                new_appetite = max(new_appetite, 0.0)
                new_pending = max(new_pending, 0.0)

                # Update the temp purchaser
                temp_demand.loc[
                    temp_demand['Purchaser Name'] == purchaser['Purchaser Name'],
                    'remaining_appetite'
                ] = new_appetite

                temp_demand.loc[
                    temp_demand['Purchaser Name'] == purchaser['Purchaser Name'],
                    'pending_amount'
                ] = new_pending

                # Record the allocation
                temp_allocations.append({
                    "Purchaser Name": purchaser['Purchaser Name'],
                    "Purchaser State": purchaser['State'],
                    "Purchaser Date": purchaser['Purchaser_Date'],
                    "Purchaser Category": purchaser['Purchaser Category'],
                    "Project ID": chunk.get('Project ID', ''),
                    "Project Name": project_name,
                    "Project Date": chunk['Project_date'],
                    "Project Category": chunk['Project Category'],
                    "Project State": chunk['Project State'],
                    "GDF Profit": chunk['GDF Profit'],
                    "Priority": chunk['Priority'],
                    "Allocated Amount": chunk_cost,
                    "Financed Amount": 0.0,  # No financing
                    "Remaining Appetite After": new_appetite,
                    "Original Appetite": purchaser['approximated_appetite']
                })

                # Mark chunk as allocated
                supply_df.loc[supply_df['Project Name'] == project_name, 'Fully Allocated'] = True
                allocated = True
                break  # Move to the next chunk after successful allocation

            # If we failed to allocate this chunk to any purchaser, skip the project
            if not allocated:
                can_allocate_project = False
                break

        # If every chunk is allocated, we finalize the temp updates
        if can_allocate_project:
            allocations.extend(temp_allocations)
            total_allocated += project_total_cost
            demand_df = temp_demand  # Make the changes permanent
            # Mark all chunks in this group as fully allocated
            supply_df.loc[group.index, 'Fully Allocated'] = True

    return total_allocated, allocations, supply_df, demand_df

# ------------------------------------------------
# 4. Main Execution
# ------------------------------------------------
def main():
    # 4.1 Read data
    supply_df = fetch_supply()
    demand_df = fetch_demand()

    # 4.2 Run allocation without financing
    total_allocated, allocations, remaining_supply, remaining_demand_df = match_supply_to_demand(
        supply_df,
        demand_df
    )

    # 4.3 Calculate summaries
    original_total_appetite = demand_df['approximated_appetite'].sum()
    remaining_demand = remaining_demand_df['remaining_appetite'].sum()

    original_total_project_value = supply_df['Original Est Cash Amount'].sum()
    remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()

    # Calculate percentage of allocated supply as a decimal
    percent_allocated_supply = (total_allocated / original_total_project_value) if original_total_project_value > 0 else 0

    # 4.4 Print Results Summary in Desired Format
    print("\nResults Summary:")
    print(f"Total Allocated \t {total_allocated:,.2f}")
    print(f"DEMAND\t SUM")
    print(f"Original Total Appetite\t {original_total_appetite:,.2f}")
    print(f"Remaining Demand\t {remaining_demand:,.2f}")
    print()
    print(f"% of Allocated Supply\t{percent_allocated_supply:.2f}")
    print(f"SUPPLY\t SUM")
    print(f"Original Total Project Value\t {original_total_project_value:,.2f}")
    print(f"Remaining Supply\t {remaining_supply_value:,.2f}")

    # 4.5 Create Supply Summary
    supply_summary = supply_df.groupby('Project Name').agg(
        Total_Chunks_Before=('Est Cash Amount', 'count'),
        Total_Chunks_After=('Fully Allocated', lambda x: x.sum()),
        Total_Project_Value=('Est Cash Amount', 'sum'),
        Latest_Project_Date=('Project_date', 'max'),           # New Column
        Max_Est_Cash_Amount=('Est Cash Amount', 'max'),       # New Column
        GDF_Profit=('GDF Profit', 'max'),                      # New Column
        Priority=('Priority', 'max')                            # Include Priority in summary
    ).reset_index()

    # ------------------------------------------------
    # **NEW:** Calculate Gross Profit for Supply Summary
    # ------------------------------------------------
    supply_summary['Gross_Profit'] = supply_summary['Total_Project_Value'] * supply_summary['GDF_Profit']

    # ------------------------------------------------
    # **NEW:** Calculate Gross Profit Metrics for Summary
    # ------------------------------------------------
    total_gross_profit = supply_summary['Gross_Profit'].sum()
    total_gross_profit_allocated = supply_summary[supply_summary['Total_Chunks_After'] > 0]['Gross_Profit'].sum()
    total_gross_profit_unallocated = supply_summary[supply_summary['Total_Chunks_After'] == 0]['Gross_Profit'].sum()

    # 4.6 Create Demand Summary (Renamed to Remaining Demand)
    allocations_df = pd.DataFrame(allocations)

    if not allocations_df.empty:
        # Group allocations by purchaser
        allocations_grouped = allocations_df.groupby('Purchaser Name').agg(
            Original_Appetite=('Original Appetite', 'first'),
            Total_Allocated=('Allocated Amount', 'sum'),
            Remaining_Appetite=('Remaining Appetite After', 'last'),
            Purchaser_Date=('Purchaser Date', 'first'),  # Use the latest date
            Purchaser_Category=('Purchaser Category', 'first')  # Include Purchaser Category
        ).reset_index()
    else:
        # If no allocations were made, create an empty grouped DataFrame
        allocations_grouped = pd.DataFrame(columns=[
            'Purchaser Name', 'Original_Appetite', 'Total_Allocated',
            'Remaining_Appetite', 'Purchaser_Date', 'Purchaser_Category'
        ])

    # Merge with original demand to include purchasers with no allocations
    demand_summary = demand_df[['Purchaser Name', 'approximated_appetite', 'pending_amount', 'Purchaser_Date', 'Purchaser Category']].rename(
        columns={
            'approximated_appetite': 'Original_Appetite',
            'pending_amount': 'Updated_Pending_Amount_Original',
            'Purchaser_Date': 'Purchaser_Date_original'  # Temporary column for merging
        }
    )

    # Perform merge with allocations_grouped to include allocation details
    demand_summary = demand_summary.merge(
        allocations_grouped[['Purchaser Name', 'Total_Allocated',
                             'Remaining_Appetite', 'Purchaser_Date', 'Purchaser_Category']],
        on='Purchaser Name',
        how='left',
        suffixes=('_original', '_alloc')
    )

    # Fill NaN for purchasers with no allocations
    demand_summary['Total_Allocated'] = demand_summary['Total_Allocated'].fillna(0.0)
    demand_summary['Remaining_Appetite'] = demand_summary['Remaining_Appetite'].fillna(demand_summary['Original_Appetite'])

    # Fill 'Purchaser Category' with allocation's category or original
    demand_summary['Purchaser Category'] = demand_summary['Purchaser_Category'].fillna(demand_summary['Purchaser Category'])

    # Handle Purchaser_Date by prioritizing allocations and filling with original dates
    demand_summary['Purchaser_Date'] = demand_summary['Purchaser_Date'].fillna(demand_summary['Purchaser_Date_original'])

    # Drop redundant 'Purchaser_Date_original', 'Updated_Pending_Amount_Original', and 'Purchaser_Category'
    demand_summary = demand_summary.drop(columns=['Purchaser_Date_original', 'Updated_Pending_Amount_Original', 'Purchaser_Category'])

    # Calculate percentages
    demand_summary['Percentage_Allocated'] = (
        (demand_summary['Total_Allocated'] / demand_summary['Original_Appetite']) * 100
    ).round(2)
    demand_summary['Percentage_Remaining'] = (
        (demand_summary['Remaining_Appetite'] / demand_summary['Original_Appetite']) * 100
    ).round(2)

    # 4.7 Create Summary DataFrame in Desired Format
    summary_data = pd.DataFrame([
        {"Metric": "Total Allocated", "Value": total_allocated},
        {"Metric": "DEMAND", "Value": "SUM"},
        {"Metric": "Original Total Appetite", "Value": original_total_appetite},
        {"Metric": "Remaining Demand", "Value": remaining_demand},
        {"Metric": "", "Value": ""},
        {"Metric": "% of Allocated Supply", "Value": percent_allocated_supply},
        {"Metric": "SUPPLY", "Value": "SUM"},
        {"Metric": "Original Total Project Value", "Value": original_total_project_value},
        {"Metric": "Remaining Supply", "Value": remaining_supply_value},
        {"Metric": "", "Value": ""},
        # **NEW:** Gross Profit Metrics
        {"Metric": "Total Gross Profit (all projects)", "Value": total_gross_profit},
        {"Metric": "Total Gross Profit Allocated", "Value": total_gross_profit_allocated},
        {"Metric": "Total Gross Profit Unallocated", "Value": total_gross_profit_unallocated}
    ])

    # 4.8 Save everything to Excel
    with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
        # Allocations Sheet
        allocations_df.to_excel(writer, sheet_name="Allocations", index=False)

        # Summary Sheet
        summary_data.to_excel(writer, sheet_name="Summary", index=False)

        # Remaining Supply Sheet
        remaining_supply_cleaned = remaining_supply.drop(columns=['Original Est Cash Amount'])
        remaining_supply_cleaned.to_excel(writer, sheet_name="Remaining Supply", index=False)

        # Remaining Demand Sheet (Renamed from Demand Summary)
        remaining_demand_cleaned = demand_summary[['Purchaser Name', 'Purchaser Category', 'Original_Appetite', 'Total_Allocated', 'Remaining_Appetite', 'Purchaser_Date', 'Percentage_Allocated', 'Percentage_Remaining']]
        remaining_demand_cleaned.to_excel(writer, sheet_name="Remaining Demand", index=False)

        # Supply Summary Sheet
        supply_summary.to_excel(writer, sheet_name="Supply Summary", index=False)

    print("Results saved to allocation_results.xlsx")

# Execute the main function
if __name__ == "__main__":
    main()



Results Summary:
Total Allocated 	 76,186,540.00
DEMAND	 SUM
Original Total Appetite	 115,581,396.00
Remaining Demand	 39,259,714.00

% of Allocated Supply	0.45
SUPPLY	 SUM
Original Total Project Value	 170,370,975.38
Remaining Supply	 49,008,529.63
Results saved to allocation_results.xlsx


# **Bank Credit Full Project Allocation**

Adjustments to Financing tab only showing completed projects

In [8]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

# -------------------------------------------------------------------------
# 0. Authenticate and open the Google Sheet
# -------------------------------------------------------------------------
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

title = "2025 Project Agenda"
sheet = gc.open(title)

# -------------------------------------------------------------------------
# 1. Fetch Supply
# -------------------------------------------------------------------------
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Keep only rows marked 'Y'
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y'].copy()

    # Convert numeric fields
    supply_df['Est Cash Amount'] = pd.to_numeric(
        supply_df['Est Cash Amount'].replace({',': ''}, regex=True),
        errors='coerce'
    )
    supply_df['Project_date'] = pd.to_datetime(
        supply_df['Project_date'], errors='coerce'
    )

    # Convert GDF Profit to numeric (e.g., "25%" -> 0.25)
    supply_df['GDF Profit'] = (
        supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    )
    supply_df = supply_df.dropna()

    # Ensure we have a Project ID
    if 'Project ID' not in supply_df.columns or supply_df['Project ID'].isnull().all():
        supply_df['Project ID'] = supply_df['Project Name']
    else:
        supply_df['Project ID'] = supply_df['Project ID'].fillna(supply_df['Project Name'])

    # If needed, derive "Project Category" from chunk size
    if 'Project Category' not in supply_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        supply_df['Project Category'] = pd.cut(
            supply_df['Est Cash Amount'], bins=bins, labels=labels
        )

    # Rename "State" -> "Project State" if needed
    if 'State' in supply_df.columns:
        supply_df.rename(columns={'State':'Project State'}, inplace=True)

    # Ensure Priority is numeric
    if 'Priority' not in supply_df.columns:
        raise ValueError("Supply data must contain a 'Priority' column.")

    supply_df['Priority'] = pd.to_numeric(supply_df['Priority'], errors='coerce')
    supply_df = supply_df.dropna(subset=['Priority'])

    # Sort supply similarly to your original logic
    supply_df = supply_df.sort_values(
        by=['Priority', 'GDF Profit', 'Est Cash Amount', 'Project_date'],
        ascending=[True, False, False, True]
    )

    # Keep an original copy of Est Cash Amount
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']

    # Add 'Fully Allocated' flag
    supply_df['Fully Allocated'] = False

    return supply_df

# -------------------------------------------------------------------------
# 2. Fetch Demand
# -------------------------------------------------------------------------
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Rename 'purchaser_name' to 'Purchaser Name' for consistency
    demand_df = demand_df.rename(columns={'purchaser_name': 'Purchaser Name'})

    # Convert numeric fields
    demand_df['approximated_appetite'] = pd.to_numeric(
        demand_df['approximated_appetite'].replace({',':''}, regex=True),
        errors='coerce'
    )

    # Drop 'pending_amount' if it exists
    if 'pending_amount' in demand_df.columns:
        demand_df.drop(columns=['pending_amount'], inplace=True)

    # Convert dates
    demand_df['Purchaser_Date'] = pd.to_datetime(demand_df['Purchaser_Date'], errors='coerce')
    demand_df = demand_df.dropna(subset=['approximated_appetite','Purchaser_Date'])

    # Track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite'].astype(float)

    # Rename "State" -> "Purchaser State" if needed
    if 'State' in demand_df.columns:
        demand_df.rename(columns={'State':'Purchaser State'}, inplace=True)

    # Ensure "Purchaser Category"
    if 'Purchaser Category' not in demand_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        demand_df['Purchaser Category'] = pd.cut(
            demand_df['approximated_appetite'], bins=bins, labels=labels
        )

    # State Priority: CA=1, others=2
    demand_df['State Priority'] = demand_df['Purchaser State'].apply(
        lambda x: 1 if x == 'CA' else 2
    )

    # Sort demand by appetite desc, then state priority asc
    demand_df = demand_df.sort_values(
        by=['approximated_appetite', 'State Priority'],
        ascending=[False, True]
    )

    return demand_df

# -------------------------------------------------------------------------
# 3. Allocate function (with optional bank bridging)
# -------------------------------------------------------------------------
def allocate_chunks_no_partial(
    supply_df: pd.DataFrame,
    demand_df: pd.DataFrame,
    bank_credit_limit: float = 20_000_000
):
    """
    Allocates supply to demand. If bank_credit_limit=0 => no bridging,
    matching the non-financing approach. If >0 => attempt bridging.
    Ensures that only fully allocated projects have their financing details recorded.
    """
    supply_df = supply_df.copy()
    demand_df = demand_df.copy()

    demand_records = demand_df.to_dict('records')
    bank_available = bank_credit_limit

    allocations = []
    financed_chunks = []
    allocated_projects = set()

    # 1) Find a direct purchaser => largest appetite + CA-first
    def find_direct_purchaser(project_date, chunk_cost, d_records):
        eligible = [
            (p, i) for i, p in enumerate(d_records)
            if p['Purchaser_Date'] <= project_date and p['remaining_appetite'] >= chunk_cost
        ]
        if not eligible:
            return None
        # Sort by (State Priority asc, remaining_appetite desc)
        eligible.sort(key=lambda x: (x[0]['State Priority'], -x[0]['remaining_appetite']))
        return eligible[0]

    # 2) Find a future purchaser for bridging => earliest date >= project date
    def find_future_purchaser_earliest_date(project_date, chunk_cost, d_records):
        future = [
            (p, i) for i, p in enumerate(d_records)
            if p['Purchaser_Date'] >= project_date and p['remaining_appetite'] >= chunk_cost
        ]
        if not future:
            return None
        # Sort by earliest Purchaser_Date
        future.sort(key=lambda x: x[0]['Purchaser_Date'])
        return future[0]

    # Helper to re-sort demand back to original logic (appetite desc, CA-first):
    def resort_demand_to_original(d_records):
        d_records.sort(key=lambda x: (-x['approximated_appetite'], x['State Priority']))
        return d_records

    # Group supply by Project Name in the same order they appear
    grouped_supply = supply_df.groupby('Project Name', sort=False)

    # ---------------------------------------------------------------------
    # Allocation loop
    # ---------------------------------------------------------------------
    for project_name, group in grouped_supply:
        if group['Fully Allocated'].all():
            continue

        project_total_cost = group['Est Cash Amount'].sum()
        project_chunks = group.to_dict('records')
        can_allocate_project = True
        temp_allocs = []
        temp_financed = []
        local_demand = [dict(d) for d in demand_records]  # Deep copy
        local_bank = bank_available

        # Sort chunks internally by (priority asc, chunk cost desc)
        sorted_chunks = sorted(
            project_chunks,
            key=lambda x: (x['Priority'], -x['Est Cash Amount'])
        )

        # For each chunk
        for chunk in sorted_chunks:
            if chunk.get('Fully Allocated'):
                continue  # Already allocated

            chunk_cost = chunk['Est Cash Amount']
            p_name  = chunk['Project Name']
            p_cat   = chunk.get('Project Category','')
            p_st    = chunk.get('Project State','')
            p_gdf   = chunk['GDF Profit']
            p_id    = chunk['Project ID']
            p_pri   = chunk['Priority']
            p_date  = chunk['Project_date']

            # Default record with "false" result
            allocation_record = {
                "Chunk Est Amount": chunk_cost,  # store the full chunk cost
                "Purchaser Name": None,
                "Purchaser State": None,
                "Purchaser Date": None,
                "Purchaser Category": None,

                "Project Name": p_name,
                "Project Date": p_date,
                "Project Category": p_cat,
                "Project State": p_st,
                "GDF Profit": p_gdf,
                "Priority": p_pri,

                "Allocated Amount": 0.0,
                "Financed Amount": 0.0,
                "Remaining Appetite After": None,
                "Original Appetite": None,

                "Project ID": p_id,
                "Result": False
            }

            # 1) Attempt direct allocation
            direct = find_direct_purchaser(p_date, chunk_cost, local_demand)
            if direct:
                pur, pur_idx = direct
                pur['remaining_appetite'] -= chunk_cost

                allocation_record.update({
                    "Purchaser Name": pur['Purchaser Name'],
                    "Purchaser State": pur.get('Purchaser State',''),
                    "Purchaser Date": pur['Purchaser_Date'],
                    "Purchaser Category": pur.get('Purchaser Category',''),
                    "Allocated Amount": chunk_cost,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": pur['remaining_appetite'],
                    "Original Appetite": pur['approximated_appetite'],
                    "Result": True
                })
                temp_allocs.append(allocation_record)
                # Mark chunk as allocated
                chunk['Fully Allocated'] = True
                continue

            # 2) No direct buyer => Attempt Bank bridging
            if local_bank >= chunk_cost and bank_credit_limit > 0:
                future = find_future_purchaser_earliest_date(p_date, chunk_cost, local_demand)
                if future:
                    f_pur, f_idx = future

                    # Bank "entry"
                    local_bank -= chunk_cost
                    bank_in = allocation_record.copy()
                    bank_in.update({
                        "Purchaser Name": "Bank Credit",
                        "Purchaser State": "N/A",
                        "Purchaser Date": p_date,
                        "Purchaser Category": "N/A",

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": chunk_cost,  # Positive indicates financing
                        "Remaining Appetite After": 0.0,
                        "Original Appetite": 0.0,
                        "Result": True
                    })
                    temp_allocs.append(bank_in)

                    # Bank "exit" to earliest-date future purchaser
                    f_pur['remaining_appetite'] -= chunk_cost
                    exit_date = f_pur['Purchaser_Date']
                    local_bank += chunk_cost

                    bank_out = allocation_record.copy()
                    bank_out.update({
                        "Purchaser Name": f_pur['Purchaser Name'],
                        "Purchaser State": f_pur.get('Purchaser State',''),
                        "Purchaser Date": exit_date,
                        "Purchaser Category": f_pur.get('Purchaser Category',''),

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": -chunk_cost,  # Negative indicates exit from bank
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],
                        "Result": True
                    })
                    temp_allocs.append(bank_out)

                    # Record bridging info
                    temp_financed.append({
                        "Project ID": p_id,
                        "Project Name": p_name,
                        "Chunk Cost": chunk_cost,
                        "Purchase Date": p_date,
                        "Exit Date": exit_date,
                        "Exit Purchaser Name": f_pur['Purchaser Name'],
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],
                        "GDF Profit": p_gdf
                    })

                    # Re-sort local_demand
                    local_demand = resort_demand_to_original(local_demand)
                else:
                    # Bridging not possible => fail the project
                    can_allocate_project = False
                    break
            else:
                # No direct buyer & bridging not possible => fail
                can_allocate_project = False
                break

        # ---------------------------------------------------------------------
        # Decide if the project was fully allocated
        # ---------------------------------------------------------------------
        if can_allocate_project:
            # Mark as allocated
            allocated_projects.add(project_name)

            # Commit local changes
            demand_records = local_demand
            bank_available = local_bank

            # Add to final allocations
            allocations.extend(temp_allocs)
            financed_chunks.extend(temp_financed)

            # Mark supply "Fully Allocated"
            supply_df.loc[group.index, 'Fully Allocated'] = True
        else:
            # If the project was not fully allocated,
            # we revert all temp_allocs for that group by adding "false" records
            for ch in group.to_dict('records'):
                chunk_cost = ch['Est Cash Amount']
                p_gdf = ch['GDF Profit']
                allocation_record_false = {
                    "Chunk Est Amount": chunk_cost,
                    "Purchaser Name": None,
                    "Purchaser State": None,
                    "Purchaser Date": None,
                    "Purchaser Category": None,
                    "Project Name": ch['Project Name'],
                    "Project Date": ch['Project_date'],
                    "Project Category": ch.get('Project Category',''),
                    "Project State": ch.get('Project State',''),
                    "GDF Profit": p_gdf,
                    "Priority": ch['Priority'],
                    "Allocated Amount": 0.0,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": None,
                    "Original Appetite": None,
                    "Project ID": ch['Project ID'],
                    "Result": False
                }
                allocations.append(allocation_record_false)
            # Do not append temp_financed since project allocation failed

        # Note: Projects that fail to allocate fully do not have their financing details recorded

    # ---------------------------------------------------------------------
    # Build Allocations DataFrame
    # ---------------------------------------------------------------------
    allocations_df = pd.DataFrame(allocations)
    alloc_cols = [
        "Purchaser Name","Purchaser State","Purchaser Date","Purchaser Category",
        "Project Name","Project Date","Project Category","Project State","GDF Profit",
        "Priority","Chunk Est Amount","Allocated Amount","Financed Amount",
        "Remaining Appetite After","Original Appetite","Project ID","Result"
    ]
    # Keep only the columns that exist
    allocations_df = allocations_df[[c for c in alloc_cols if c in allocations_df.columns]]

    # IMPORTANT: This column now represents the "theoretical" gross profit for *each chunk*
    # regardless of whether Result=True or False.
    # i.e. chunk_est_amount * GDF_Profit
    allocations_df['Chunk Gross Profit'] = (
        allocations_df['Chunk Est Amount'] * allocations_df['GDF Profit']
    )

    # ---------------------------------------------------------------------
    # Build Financing DataFrame (only fully allocated projects)
    # ---------------------------------------------------------------------
    financed_df = pd.DataFrame(financed_chunks)
    fin_cols = [
        "Project ID","Project Name","Chunk Cost","Purchase Date","Exit Date",
        "Exit Purchaser Name","Remaining Appetite After","Original Appetite",
        "GDF Profit"
    ]
    for cc in fin_cols:
        if cc not in financed_df.columns:
            financed_df[cc] = None
    financed_df = financed_df[fin_cols]

    # If bridging was used, calculate interest & profit
    if not financed_df.empty:
        financed_df['Purchase Date'] = pd.to_datetime(financed_df['Purchase Date'], errors='coerce')
        financed_df['Exit Date'] = pd.to_datetime(financed_df['Exit Date'], errors='coerce')

        # Months of bank credit
        financed_df['Time of Bank Credit'] = financed_df.apply(
            lambda row: (
                (row['Exit Date'].year - row['Purchase Date'].year) * 12
                + (row['Exit Date'].month - row['Purchase Date'].month)
            ),
            axis=1
        )

        # 1% monthly interest
        financed_df['Cost of Interest'] = financed_df['Chunk Cost'] * 0.01 * financed_df['Time of Bank Credit']

        financed_df['Gross Profit'] = financed_df['Chunk Cost'] * financed_df['GDF Profit']
        financed_df['Net Profit']   = financed_df['Gross Profit'] - financed_df['Cost of Interest']
        financed_df['Net profit %'] = financed_df['Net Profit'] / financed_df['Chunk Cost']
    else:
        financed_df['Time of Bank Credit'] = None
        financed_df['Cost of Interest'] = None
        financed_df['Gross Profit'] = None
        financed_df['Net Profit'] = None
        financed_df['Net profit %'] = None

    # ---------------------------------------------------------------------
    # Summary Metrics
    # ---------------------------------------------------------------------
    # Total allocated (exclude the temporary "Bank Credit" allocations)
    total_alloc = (
        allocations_df['Allocated Amount'].sum()
        - allocations_df.loc[allocations_df['Purchaser Name'] == 'Bank Credit', 'Allocated Amount'].sum()
    )
    financed_used = allocations_df.loc[allocations_df['Financed Amount'] > 0, 'Financed Amount'].sum()
    bank_remaining = bank_available

    # Demand side
    original_total_appetite = demand_df['approximated_appetite'].sum()
    final_demand_records = pd.DataFrame(demand_records)
    remaining_demand = final_demand_records['remaining_appetite'].sum()

    # Supply side
    original_total_project_value = supply_df['Original Est Cash Amount'].sum()
    unallocated_projects = set(supply_df['Project Name'].unique()) - allocated_projects
    unallocated_supply_df = supply_df[supply_df['Project Name'].isin(unallocated_projects)]
    remaining_supply_value = unallocated_supply_df['Est Cash Amount'].sum()

    pct_alloc = 0.0
    if original_total_project_value > 0:
        pct_alloc = (total_alloc / original_total_project_value) * 100

    # Prepare a high-level summary DataFrame
    summary_data = [
        {"Metric": "--- RESULTS SUMMARY ---", "Value": ""},
        {"Metric": "Total Allocated", "Value": f"{total_alloc:,.2f}"},
        {"Metric": "Financed (Bank Used)", "Value": f"{financed_used:,.2f}"},
        {"Metric": "Bank Credit Remaining", "Value": f"{bank_remaining:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "DEMAND SUM", "Value": ""},
        {"Metric": "Original Total Appetite", "Value": f"{original_total_appetite:,.2f}"},
        {"Metric": "Remaining Demand", "Value": f"{remaining_demand:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "% of Allocated Supply", "Value": f"{pct_alloc:.2f}%"},

        {"Metric": "", "Value": ""},
        {"Metric": "SUPPLY SUM", "Value": ""},
        {"Metric": "Original Total Project Value", "Value": f"{original_total_project_value:,.2f}"},
        {"Metric": "Remaining Supply", "Value": f"{remaining_supply_value:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "--- GROSS PROFIT METRICS ---", "Value": ""},
        {"Metric": "Total Gross Profit (theoretical all)", "Value": ""},  # -3
        {"Metric": "Total Gross Profit (allocated)", "Value": ""},        # -2
        {"Metric": "Total Gross Profit (unallocated)", "Value": ""}       # -1
    ]
    summary_df = pd.DataFrame(summary_data)

    # ---------------------------------------------------------------------
    # Supply Summary
    # ---------------------------------------------------------------------
    # Mark final Results per project (if all chunks got allocated => True)
    project_results = allocations_df.groupby('Project Name')['Result'].all().reset_index()
    project_results.rename(columns={'Result': 'Fully_Allocated'}, inplace=True)

    supply_after_df = supply_df.merge(project_results, on='Project Name', how='left')
    supply_after_df['Fully_Allocated'] = supply_after_df['Fully_Allocated'].fillna(False)

    # Basic aggregator
    supply_summary = supply_after_df.groupby('Project Name').agg(
        Total_Chunks_Before=('Est Cash Amount', 'count'),
        Total_Project_Value=('Est Cash Amount', 'sum'),
        Latest_Project_Date=('Project_date', 'max'),
        Max_Est_Cash_Amount=('Est Cash Amount', 'max'),
        GDF_Profit=('GDF Profit', 'max'),  # just for reference
        Priority=('Priority', 'max'),
        Fully_Allocated=('Fully_Allocated', 'max')  # To determine allocation status
    ).reset_index()

    # 1) Theoretical GP = sum of chunk_gross_profit for all chunks (excluding Bank Credit)
    gp_theoretical = (
        allocations_df
        .loc[allocations_df['Purchaser Name'] != 'Bank Credit']
        .groupby('Project Name')['Chunk Gross Profit']
        .sum()
        .reset_index(name='Gross_Profit_Theoretical')
    )

    # 2) Allocated GP = Gross_Profit_Theoretical if fully allocated, else 0
    # Corrected the column name from 'Fully Allocated' to 'Fully_Allocated'
    gp_allocated = gp_theoretical.copy()
    gp_allocated['Gross_Profit_Allocated'] = np.where(
        gp_allocated['Project Name'].isin(supply_summary.loc[supply_summary['Fully_Allocated'], 'Project Name']),
        gp_allocated['Gross_Profit_Theoretical'],
        0
    )
    gp_allocated = gp_allocated[['Project Name', 'Gross_Profit_Allocated']]

    # 3) Unallocated GP = 0 if fully allocated, else Gross_Profit_Theoretical
    gp_unallocated = gp_theoretical.copy()
    gp_unallocated['Gross_Profit_Unallocated'] = np.where(
        gp_unallocated['Project Name'].isin(supply_summary.loc[supply_summary['Fully_Allocated'], 'Project Name']),
        0,
        gp_unallocated['Gross_Profit_Theoretical']
    )
    gp_unallocated = gp_unallocated[['Project Name', 'Gross_Profit_Unallocated']]

    # Merge into supply_summary
    supply_summary = supply_summary.merge(gp_theoretical, on='Project Name', how='left')
    supply_summary = supply_summary.merge(gp_allocated, on='Project Name', how='left')
    supply_summary = supply_summary.merge(gp_unallocated, on='Project Name', how='left')

    supply_summary['Gross_Profit_Theoretical'] = supply_summary['Gross_Profit_Theoretical'].fillna(0)
    supply_summary['Gross_Profit_Allocated']   = supply_summary['Gross_Profit_Allocated'].fillna(0)
    supply_summary['Gross_Profit_Unallocated'] = supply_summary['Gross_Profit_Unallocated'].fillna(0)

    # Totals for summary
    total_gross_profit_theoretical = supply_summary['Gross_Profit_Theoretical'].sum()
    total_gross_profit_allocated   = supply_summary['Gross_Profit_Allocated'].sum()
    total_gross_profit_unallocated = supply_summary['Gross_Profit_Unallocated'].sum()

    # Fill these into summary_df
    summary_df.iloc[-3, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_theoretical:,.2f}"
    summary_df.iloc[-2, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_allocated:,.2f}"
    summary_df.iloc[-1, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_unallocated:,.2f}"

    # ---------------------------------------------------------------------
    # Demand After
    # ---------------------------------------------------------------------
    updated_demand_df = pd.DataFrame(demand_records).copy()
    updated_demand_df['allocated_amount'] = (
        updated_demand_df['approximated_appetite'] - updated_demand_df['remaining_appetite']
    )
    updated_demand_df["Financing"] = 0.0

    # If bridging was used, track how much each purchaser financed
    if not allocations_df.empty and "Financed Amount" in allocations_df.columns:
        financing_by_purchaser = (
            allocations_df[allocations_df['Financed Amount'] < 0]
            .groupby('Purchaser Name')['Financed Amount']
            .sum()
            .abs()
            .reset_index(name='Financing')
        )
        updated_demand_df = updated_demand_df.merge(
            financing_by_purchaser, on='Purchaser Name', how='left', suffixes=('', '_calc')
        )
        updated_demand_df['Financing'] = updated_demand_df['Financing_calc'].fillna(0)
        updated_demand_df.drop(columns=['Financing_calc'], inplace=True)

    demand_after_cols = [
        "Purchaser Name",
        "approximated_appetite",
        "allocated_amount",
        "Purchaser State",
        "Purchaser_Date",
        "Financing",
        "remaining_appetite",
        "Purchaser Category",
        "State Priority"
    ]
    for c in demand_after_cols:
        if c not in updated_demand_df.columns:
            updated_demand_df[c] = None

    updated_demand_df = updated_demand_df[demand_after_cols]

    return (
        allocations_df,      # 1) Detailed chunk-by-chunk allocations (with full "Chunk Est Amount")
        financed_df,         # 2) Bank bridging details (only fully allocated projects)
        summary_df,          # 3) High-level summary
        supply_summary,      # 4) Per-project supply summary (now chunk-based theoretical GP)
        supply_after_df,     # 5) Supply with final project 'Result'
        updated_demand_df    # 6) Demand after final allocations
    )

# -------------------------------------------------------------------------
# 4. Main Execution
# -------------------------------------------------------------------------
def main():
    supply_df = fetch_supply()
    demand_df = fetch_demand()

    (
        allocations_df,
        financed_df,
        summary_df,
        supply_summary,
        updated_supply,
        updated_demand_df
    ) = allocate_chunks_no_partial(
        supply_df,
        demand_df,
        bank_credit_limit=20_000_000
    )

    # Print the summary to console
    print(summary_df.to_string(index=False))

    # Save to Excel in multiple tabs
    with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
        allocations_df.to_excel(writer, sheet_name="Allocations", index=False)
        financed_df.to_excel(writer, sheet_name="Financing Chunks", index=False)
        summary_df.to_excel(writer, sheet_name="Summary", index=False)
        supply_summary.to_excel(writer, sheet_name="Supply Summary", index=False)
        updated_supply.to_excel(writer, sheet_name="SupplyAfter", index=False)
        updated_demand_df.to_excel(writer, sheet_name="DemandAfter", index=False)

    print("\nResults saved to 'allocation_results.xlsx' with:")
    print(" - Allocations (now with 'Chunk Est Amount' and 'Chunk Gross Profit' for TRUE or FALSE)")
    print(" - Financing Chunks (details of bridging, interest & profit columns; only fully allocated projects)")
    print(" - Summary (result metrics including chunk-based total GP)")
    print(" - Supply Summary (per-project chunk-based GP: theoretical vs allocated vs unallocated)")
    print(" - SupplyAfter (indicates overall project success/failure in 'Fully_Allocated')")
    print(" - DemandAfter (final states of demand)")

if __name__ == "__main__":
    main()

                              Metric          Value
             --- RESULTS SUMMARY ---               
                     Total Allocated  77,491,858.23
                Financed (Bank Used)   1,062,527.87
               Bank Credit Remaining  20,000,000.00
                                                   
                          DEMAND SUM               
             Original Total Appetite 115,441,395.00
                    Remaining Demand  37,949,536.77
                                                   
               % of Allocated Supply         45.48%
                                                   
                          SUPPLY SUM               
        Original Total Project Value 170,370,975.38
                    Remaining Supply  92,879,117.15
                                                   
        --- GROSS PROFIT METRICS ---               
Total Gross Profit (theoretical all)  43,186,740.39
      Total Gross Profit (allocated)  19,248,019.82
    Total Gr

# **Financing Matrix / Cost of Interest**

In [ ]:
import pandas as pd
import numpy as np

# Load the Excel file and Financing Chunks tab
file_path = '/content/allocation_results.xlsx'
financing_chunks_df = pd.read_excel(file_path, sheet_name='Financing Chunks')

# Ensure date columns are in datetime format
financing_chunks_df['Purchase Date'] = pd.to_datetime(financing_chunks_df['Purchase Date'])
financing_chunks_df['Exit Date'] = pd.to_datetime(financing_chunks_df['Exit Date'])

# Create a matrix of active financing months
months = list(range(1, 13))  # Months from January to December
active_months_matrix = pd.DataFrame(0, index=financing_chunks_df.index, columns=months)

# Populate the matrix with 1s where financing was active
for index, row in financing_chunks_df.iterrows():
    start_month = row['Purchase Date'].month
    exit_month = row['Exit Date'].month

    # Apply the logic to mark active months
    for month in months:
        if start_month <= month < exit_month:
            active_months_matrix.loc[index, month] = 1

# Add the active months matrix to the main DataFrame
financing_chunks_df = pd.concat([financing_chunks_df, active_months_matrix], axis=1)

# Calculate the total financing amounts across active months
financing_amounts_matrix = active_months_matrix.multiply(financing_chunks_df['Chunk Cost'], axis=0)
financing_amounts_matrix['Project Name'] = financing_chunks_df['Project Name']  # Add Project Name column
financing_chunks_df['Total Financing Amount'] = financing_amounts_matrix.drop(columns=['Project Name']).sum(axis=1)

# Calculate the cost of financed projects based on interest rate
interest_rate = 0.01  # 1% monthly interest
interest_cost_matrix = financing_amounts_matrix.drop(columns=['Project Name']) * interest_rate
interest_cost_matrix['Project Name'] = financing_chunks_df['Project Name']  # Add Project Name column
financing_chunks_df['Total Interest Cost'] = interest_cost_matrix.drop(columns=['Project Name']).sum(axis=1)

# Compute the row of totals for all periods and add label
total_financing_row = financing_amounts_matrix.drop(columns=['Project Name']).sum().to_frame().T
total_financing_row['Project Name'] = "TOTALS"
total_financing_row.index = ['Total']

total_interest_row = interest_cost_matrix.drop(columns=['Project Name']).sum().to_frame().T
total_interest_row['Project Name'] = "TOTALS"
total_interest_row.index = ['Total']

# Group by Project Name and create matrices for each project (Active Financing and Interest Cost)
project_financing_matrices = {}
project_interest_matrices = {}

for project_name, group in financing_chunks_df.groupby('Project Name'):
    # Select rows for the current project from financing and interest matrices
    financing_matrix = financing_amounts_matrix.loc[group.index].drop(columns=['Project Name'])
    interest_matrix = interest_cost_matrix.loc[group.index].drop(columns=['Project Name'])

    # Create TOTALS row for the project
    project_total_financing = financing_matrix.sum().to_frame().T
    project_total_financing['Project Name'] = f"TOTALS - {project_name}"

    project_total_interest = interest_matrix.sum().to_frame().T
    project_total_interest['Project Name'] = f"TOTALS - {project_name}"

    # Append total row
    financing_matrix = pd.concat([financing_matrix, project_total_financing], ignore_index=True)
    interest_matrix = pd.concat([interest_matrix, project_total_interest], ignore_index=True)

    financing_matrix.index = list(group.index) + ['Total']
    interest_matrix.index = list(group.index) + ['Total']

    project_financing_matrices[project_name] = financing_matrix
    project_interest_matrices[project_name] = interest_matrix

# Save results to an Excel file
output_file = '/content/financing_matrices_corrected.xlsx'
with pd.ExcelWriter(output_file) as writer:
    # Save the main DataFrame
    financing_chunks_df.to_excel(writer, sheet_name='Financing Data', index=False)

    # Save the Financing Amounts matrix with totals
    financing_amounts_matrix = pd.concat([financing_amounts_matrix, total_financing_row], ignore_index=True)
    financing_amounts_matrix.to_excel(writer, sheet_name='Financing Amounts', index=False)

    # Save the Interest Costs matrix with totals
    interest_cost_matrix = pd.concat([interest_cost_matrix, total_interest_row], ignore_index=True)
    interest_cost_matrix.to_excel(writer, sheet_name='Interest Costs', index=False)

    # Save individual project matrices with sanitized sheet names
    for project_name, matrix in project_financing_matrices.items():
        sanitized_name = f'Fin_{project_name[:25]}'.replace(' ', '_')  # Truncate and replace spaces
        matrix.to_excel(writer, sheet_name=sanitized_name, index=False)

    for project_name, matrix in project_interest_matrices.items():
        sanitized_name = f'Int_{project_name[:25]}'.replace(' ', '_')  # Truncate and replace spaces
        matrix.to_excel(writer, sheet_name=sanitized_name, index=False)

print(f"Results saved to {output_file}")

Plotly of remainings

In [18]:
import pandas as pd
import plotly.express as px

# Define default chart size
DEFAULT_WIDTH = 1000
DEFAULT_HEIGHT = 700

# Load the allocation results from Excel
file_path = "allocation_results.xlsx"

# Read necessary sheets with error handling
try:
    supply_after_df = pd.read_excel(file_path, sheet_name="SupplyAfter")
    demand_after_df = pd.read_excel(file_path, sheet_name="DemandAfter")
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    exit(1)
except ValueError as e:
    print(f"Error reading Excel file: {e}")
    exit(1)

# --------------------------------------------------------------------
# 1. Plot Supply After: Bar Chart Showing Allocated vs Unallocated
#    - Includes Total Est Cash Amount, Project Count, and Est Cash Chunks
#    - "Fully Allocated" in blue, "Unallocated" in gray
#    - Saves chart as HTML
# --------------------------------------------------------------------
def plot_supply_after(supply_after_df):
    if supply_after_df.empty:
        print("SupplyAfter sheet is empty.")
        return

    # Group by allocation status and calculate sum and counts
    supply_summary = supply_after_df.groupby('Fully_Allocated').agg(
        Total_Est_Cash_Amount=('Original Est Cash Amount', 'sum'),
        Project_Count=('Project Name', 'nunique'),  # Unique projects
        Est_Cash_Chunks=('Original Est Cash Amount', 'count')  # Total chunks
    ).reset_index()

    # Map boolean to status labels
    supply_summary['Status'] = supply_summary['Fully_Allocated'].map({True: "Fully Allocated", False: "Unallocated"})

    # Create a combined text for sum, project count, and cash chunks
    supply_summary['Text'] = supply_summary.apply(
        lambda row: f"$ {row['Total_Est_Cash_Amount']:,.2f}<br>Projects: {row['Project_Count']}<br>Chunks: {row['Est_Cash_Chunks']}",
        axis=1
    )

    # Define color mapping
    color_map = {
        "Fully Allocated": "blue",
        "Unallocated": "gray"
    }

    # Create the bar chart
    fig = px.bar(
        supply_summary,
        x="Status",
        y="Total_Est_Cash_Amount",
        text="Text",
        title="Supply After: Fully Allocated vs. Unallocated Projects",
        labels={"Total_Est_Cash_Amount": "Total Project Value ($)"},
        color="Status",
        color_discrete_map=color_map,
        width=DEFAULT_WIDTH,
        height=DEFAULT_HEIGHT
    )

    # Update trace to show the custom text
    fig.update_traces(texttemplate='%{text}', textposition='outside')
    fig.update_layout(
        yaxis_title="Total Project Value ($)",
        xaxis_title="Allocation Status",
        uniformtext_minsize=8,
        uniformtext_mode='hide',
        plot_bgcolor='rgba(0,0,0,0)',  # Optional: Make background transparent
        legend=dict(title="Status")
    )

    # Display the figure
    fig.show()

    # Save the figure as HTML
    fig.write_html("supply_after_chart.html")
    print("Supply After chart saved as 'supply_after_chart.html'.")

# --------------------------------------------------------------------
# 2. Plot Demand After: Pie Chart of Remaining Demand
#    - "Total Allocated" in blue, "Remaining Demand" in gray
#    - Saves chart as HTML
# --------------------------------------------------------------------
def plot_demand_after(demand_after_df):
    if demand_after_df.empty:
        print("DemandAfter sheet is empty.")
        return

    demand_summary = {
        "Total Allocated": demand_after_df["allocated_amount"].sum(),
        "Remaining Demand": demand_after_df["remaining_appetite"].sum()
    }

    # Define color mapping
    color_map = {
        "Total Allocated": "blue",
        "Remaining Demand": "gray"
    }

    fig = px.pie(
        names=list(demand_summary.keys()),
        values=list(demand_summary.values()),
        title="Demand After: Allocated vs. Remaining Appetite",
        hole=0.4,
        color=list(demand_summary.keys()),
        color_discrete_map=color_map,
        width=DEFAULT_WIDTH,
        height=DEFAULT_HEIGHT
    )

    fig.update_traces(textinfo="percent+label", pull=[0.1, 0])
    fig.update_layout(
        legend=dict(title="Demand Status"),
        plot_bgcolor='rgba(0,0,0,0)'  # Optional: Make background transparent
    )

    # Display the figure
    fig.show()

    # Save the figure as HTML
    fig.write_html("demand_after_chart.html")
    print("Demand After chart saved as 'demand_after_chart.html'.")

# --------------------------------------------------------------------
# 3. Bar Chart of Remaining Est Cash Amount on Supply (Per Project)
#    - Includes Count of Est Cash Chunks and Total Sum of Chunks
#    - Ordered from highest to lowest
#    - Saves chart as HTML
# --------------------------------------------------------------------
def plot_supply_remaining_bar(supply_after_df):
    # Filter for unallocated supply
    remaining_supply = supply_after_df[supply_after_df['Fully_Allocated'] == False]

    if remaining_supply.empty:
        print("No unallocated supply to display.")
        return

    # Aggregate total sum and count per project
    project_summary = remaining_supply.groupby('Project Name').agg(
        Total_Est_Cash_Amount=('Original Est Cash Amount', 'sum'),
        Est_Cash_Chunks=('Original Est Cash Amount', 'count')
    ).reset_index()

    # Merge the summary back with the remaining_supply to have sum and count per row
    remaining_supply_merged = remaining_supply.merge(project_summary, on='Project Name', how='left')

    # Sort the data in descending order of Original Est Cash Amount
    remaining_supply_sorted = remaining_supply_merged.sort_values(by='Original Est Cash Amount', ascending=False)

    # Create a combined text for each chunk including chunk size, project total, and chunk count
    remaining_supply_sorted['Text'] = remaining_supply_sorted.apply(
        lambda row: (
            f"Chunk Size: $ {row['Original Est Cash Amount']:,.2f}<br>"
            f"Project Total: $ {row['Total_Est_Cash_Amount']:,.2f}<br>"
            f"Total Chunks: {row['Est_Cash_Chunks']}"
        ),
        axis=1
    )

    fig = px.bar(
        remaining_supply_sorted,
        x="Original Est Cash Amount",
        y="Project Name",
        text="Text",
        title="Remaining Est Cash Amount per Project (Unallocated Supply)",
        labels={
            "Original Est Cash Amount": "Est Cash Amount ($)",
            "Project Name": "Project"
        },
        color="Original Est Cash Amount",
        orientation='h',  # Horizontal bars for better readability
        width=DEFAULT_WIDTH,
        height=DEFAULT_HEIGHT,
        hover_data={
            "Project Name": False,
            "Original Est Cash Amount": False,  # Already included in text
            "Total_Est_Cash_Amount": True,
            "Est_Cash_Chunks": True
        }
    )

    fig.update_traces(texttemplate='%{text}', textposition='outside')
    fig.update_layout(
        xaxis_title="Remaining Est Cash Amount ($)",
        yaxis_title="Project Name",
        yaxis=dict(autorange="reversed"),  # Ensures highest values are on top
        uniformtext_minsize=8,
        uniformtext_mode='hide',
        plot_bgcolor='rgba(0,0,0,0)',  # Optional: Make background transparent
        legend=dict(title="Est Cash Amount")
    )

    # Display the figure
    fig.show()

    # Save the figure as HTML
    fig.write_html("supply_remaining_bar_chart.html")
    print("Supply Remaining Bar Chart saved as 'supply_remaining_bar_chart.html'.")

# --------------------------------------------------------------------
# 4. Bar Chart of Remaining Appetite on Demand (Per Purchaser)
#    - Ordered from highest to lowest
#    - Saves chart as HTML
# --------------------------------------------------------------------
def plot_demand_remaining_bar(demand_after_df):
    remaining_demand = demand_after_df[demand_after_df["remaining_appetite"] > 0]

    if remaining_demand.empty:
        print("No remaining appetite to display.")
        return

    # Sort the data in descending order
    remaining_demand_sorted = remaining_demand.sort_values(by='remaining_appetite', ascending=False)

    fig = px.bar(
        remaining_demand_sorted,
        x="remaining_appetite",
        y="Purchaser Name",
        text="remaining_appetite",
        title="Remaining Appetite per Purchaser (Unfulfilled Demand)",
        labels={"remaining_appetite": "Remaining Appetite ($)", "Purchaser Name": "Purchaser"},
        color="remaining_appetite",
        orientation='h',  # Horizontal bars for better readability
        width=DEFAULT_WIDTH,
        height=DEFAULT_HEIGHT
    )

    fig.update_traces(texttemplate='$%{text:,.2f}', textposition='outside')
    fig.update_layout(
        xaxis_title="Remaining Appetite ($)",
        yaxis_title="Purchaser Name",
        yaxis=dict(autorange="reversed"),  # Ensures highest values are on top
        uniformtext_minsize=8,
        uniformtext_mode='hide',
        plot_bgcolor='rgba(0,0,0,0)',  # Optional: Make background transparent
        legend=dict(title="Remaining Appetite")
    )

    # Display the figure
    fig.show()

    # Save the figure as HTML
    fig.write_html("demand_remaining_bar_chart.html")
    print("Demand Remaining Bar Chart saved as 'demand_remaining_bar_chart.html'.")

# --------------------------------------------------------------------
# Execute the Plots
# --------------------------------------------------------------------
plot_supply_after(supply_after_df)
plot_demand_after(demand_after_df)
plot_supply_remaining_bar(supply_after_df)
plot_demand_remaining_bar(demand_after_df)


Supply After chart saved as 'supply_after_chart.html'.


Demand After chart saved as 'demand_after_chart.html'.


Supply Remaining Bar Chart saved as 'supply_remaining_bar_chart.html'.


Demand Remaining Bar Chart saved as 'demand_remaining_bar_chart.html'.
